## 验证数据集  

之前我们会对数据集做train_test_split操作，将数据集分为两份    

然后进行训练，使用测试数据的误差来评判模型的好坏，如果一次训练过后，模型的在测试集上的误差较大，就会调整超参数    
直到调整到模型在测试数据集上有较好的性能，**但其实这样会使得模型在测试集上发生过拟合**   

所以引入验证数据集，我们在数据集切分时，将数据集分为三份   

然后使用训练集训练模型，使用验证集调整超参数，将测试集作为一个模型从未见过的一个模拟真实环境(生产环境)的数据集   

## 验证数据集的意义

* 验证数据集意义   
    调整超参数使用的数据集   
   
* 测试数据集意义  
    作为衡量最终模型性能的数据集   

## 交叉验证


In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

### 先使用train_test_split的方法来调超参数

In [3]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
# 使用kNN算法来进行测试
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [4]:
# 寻找KNN算法中的两个超参数的较优值，K，计算当前样本点与周围k个点的距离，p为距离超参数
best_score, best_k, best_p = 0, 0, 0

for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score = score
            best_k = k
            best_p = p
            
print("best_score", best_score)
print("best_k", best_k)
print("best_p", best_p)

best_score 0.9860917941585535
best_k 3
best_p 4


### 使用交叉验证

![crossvalidation](./img/crossvalidation.png)

* 首先将数据集分为训练集和测试集   
* 对训练集均分成k份，上图中共分为了3份    
* 对k份数据集进行组合，比如两份做训练集，剩下一份做验证数据集   
* 这样就可以形成k种训练集和验证集的组合，每种组合都能训练出一个模型   
* 我们对这k种模型得到的结果取均值来作为当前超参数发挥效果的得分（这里面使用k个模型的均值其实已经对异常值进行了处理）    
* 如果当前这k种模型的得分并不好，就调节超参数继续进行上述过程 

In [7]:
from sklearn.model_selection import cross_val_score 
import warnings 

warnings.filterwarnings("ignore")

# 创建一个默认参数的Knn分类器
knn_clf = KNeighborsClassifier()
# 直接将要进行交叉验证的分类器传入，然后传入当前的训练集
cross_val_score(knn_clf, X_train, y_train)
# 输出是三个得分，说明在当前默认超参数的情况下，对训练数据集进行了3次切分，分成了3份，然后对3个模型分别进行了计算

array([0.98895028, 0.97777778, 0.96629213])

In [8]:
# 接下来对knn模型使用交叉验证的方式寻找最优超参数k和p
best_score, best_p, best_k = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        # 将当前超参数k和p创建的knn分类器传入交叉验证函数中
        # 并将三个模型的得分进行记录
        scores = cross_val_score(knn_clf, X_train, y_train)
        # 计算三个模型得分的均值
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_p = p
            best_k = k
            
print("best_score", best_score)
print("best_k", best_k)
print("best_p", best_p)

best_score 0.9823599874006478
best_k 2
best_p 2


In [9]:
# 通过上面的交叉验证得到的最优的k和p，下面使用这两个最优超参数训练新的模型  
knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2)
# 使用训练数据集进行训练
knn_clf.fit(X_train, y_train)
# 使用测试集进行模型评价，这个测试集模型是没有见过的
knn_clf.score(X_test, y_test)

0.980528511821975

## 回顾网格搜索